In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

In [0]:
# Read data from the Silver table
df_silver_football_matches = spark.read.table("football_matches_catalog.silver.football_matches")

In [0]:
# Display the sivler data
display(df_silver_football_matches)

In [0]:
# Compute statistics for home matches (per season, per team)
df_home_team_stats = (
    df_silver_football_matches
        .withColumnRenamed("HomeTeam", "Team")
        .groupBy("Season", "Team")
        .agg(
        sum("HomeTeamGoals").alias("GoalsScored"),
        sum("AwayTeamGoals").alias("GoalsAgainst"),
        sum(when(col("FinalResult") == "H", 1).otherwise(0)).alias("Win"),
        sum(when(col("FinalResult") == "A", 1).otherwise(0)).alias("Loss"),
        sum(when(col("FinalResult") == "D", 1).otherwise(0)).alias("Tie"),
    )
)


In [0]:
# Display the statistics for home matches
display(df_home_team_stats)

In [0]:
# Compute statistics for away matches (per season, per team)
df_away_team_stats = (
    df_silver_football_matches
        .withColumnRenamed("AwayTeam", "Team")
        .groupBy("Season", "Team")
        .agg(
        sum("AwayTeamGoals").alias("GoalsScored"),
        sum("HomeTeamGoals").alias("GoalsAgainst"),
        sum(when(col("FinalResult") == "A", 1)).alias("Win"),
        sum(when(col("FinalResult") == "H", 1)).alias("Loss"),
        sum(when(col("FinalResult") == "D", 1)).alias("Tie"),
    )
)


In [0]:
# Display the statistics for away matches
display(df_away_team_stats)

In [0]:
# Combine home and away statistics to build full team stats
df_season_team_agg_stats = (
    df_home_team_stats
    .union(df_away_team_stats)
    .groupBy("Season", "Team")
    .agg(
        sum("GoalsScored").alias("GoalsScored"),
        sum("GoalsAgainst").alias("GoalsAgainst"),
        sum("Win").alias("Win"),
        sum("Loss").alias("Loss"),
        sum("Tie").alias("Tie")
    )
)

In [0]:
# Display the full team stats
display(df_season_team_agg_stats)

In [0]:
# Add two new columns using the following logic GoalDifferentials Compute it as: GoalsScored - GoalsAgainst
# Points Compute it as: (Win * 3) + Tie
# WinPercentage Compute it as: Win / (Win + Loss + Tie)
df_season_team_enriched_stats = (
    df_season_team_agg_stats
        .withColumn("GoalDifferentials", col("GoalsScored") - col("GoalsAgainst"))
        .withColumn("Points", (col("Win") * 3) + col("Tie"))
        .withColumn("WinPercentage", round((col("Win") / (col("Win") + col("Loss") + col("Tie"))) * 100, 2)) 
)

In [0]:
# Display the enriched team stats
display(df_season_team_enriched_stats)

In [0]:
# Create a new table called df_gold_team_rankings that ranks all teams within each season based on their performance
# Define a window by season, ordering by wins then goal differential
window_spec = (Window.partitionBy("Season").orderBy(desc("Points"), desc("GoalDifferentials"), desc("Win")))

In [0]:
# Assign a position for each team in its season
df_gold_team_rankings = (df_season_team_enriched_stats.withColumn("TeamPosition", row_number().over(window_spec)))

In [0]:
# Display the gold table
display(df_gold_team_rankings)

In [0]:
# Write the final Gold-layer ranking table
df_gold_team_rankings.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable("football_matches_catalog.gold.football_matches_ranking")